#Around the World in 60 Minutes

Tired of travel restrictions? Are you feet itching to start traveling again? 

Let data take you around the world. In this exercise, we will explore a [SQL database from the World Factbook, provided by CIA](https://www.cia.gov/the-world-factbook/). Although the database is slightly outdated (it's from 2015), this virtual tour around the world will be much more environment-friendly and quench your thirst for discovering the unknown world! 

Main questions we will answer are: 
* Which countries have the most and least population on Earth?
* Which country has the highest population density? 
* Which country has higher death rate than birth rate?

##Data
###Connect with the database and load data

In [40]:
import pandas as pd
import sqlite3 as sql

In [44]:
%%capture
%load_ext sql
%sql sqlite:///5_factbook.db

In [45]:
%%sql
SELECT *
    FROM sqlite_master
    WHERE type='table';

 * sqlite:///5_factbook.db
   sqlite:///factbook.db
Done.


[('table', 'sqlite_sequence', 'sqlite_sequence', 3, 'CREATE TABLE sqlite_sequence(name,seq)'),
 ('table', 'facts', 'facts', 47, 'CREATE TABLE "facts" ("id" INTEGER PRIMARY KEY AUTOINCREMENT NOT NULL, "code" varchar(255) NOT NULL, "name" varchar(255) NOT NULL, "area" integer, "a ... (4 characters truncated) ... land" integer, "area_water" integer, "population" integer, "population_growth" float, "birth_rate" float, "death_rate" float, "migration_rate" float)')]

In [46]:
%%sql
SELECT *
    FROM facts
    LIMIT 5;

 * sqlite:///5_factbook.db
   sqlite:///factbook.db
Done.


[(1, 'af', 'Afghanistan', 652230, 652230, 0, 32564342, 2.32, 38.57, 13.89, 1.51),
 (2, 'al', 'Albania', 28748, 27398, 1350, 3029278, 0.3, 12.92, 6.58, 3.3),
 (3, 'ag', 'Algeria', 2381741, 2381741, 0, 39542166, 1.84, 23.67, 4.31, 0.92),
 (4, 'an', 'Andorra', 468, 468, 0, 85580, 0.12, 8.13, 6.96, 0.0),
 (5, 'ao', 'Angola', 1246700, 1246700, 0, 19625353, 2.78, 38.78, 11.49, 0.46)]

Relevant variables to our research questions are:
* name : name of the country;
* area : total area of the country in square kilometers;
* area_land : land area of the country in square kilometers (we will use this to calculate the population density because people cannot live on water);
* population : population as of 2015;
* birth_rate : number of births per year per 1,000 people);
* death_rate : number of deaths per year per 1,000 people.

##Explore the dataset with summary statistics

In [49]:
%%sql
SELECT MIN(population) as min_pop,
        MAX(population) as max_pop,
        MIN(population_growth) as min_pop_grt,
        MAX(population_growth) as max_pop_grt
FROM facts;


 * sqlite:///5_factbook.db
   sqlite:///factbook.db
Done.


[(0, 7256490011, 0.0, 4.02)]

In [52]:
%%sql
SELECT name, population 
FROM facts
WHERE population == (SELECT MIN(population) 
                    FROM facts) 
OR population == (SELECT MAX(population) FROM facts)

 * sqlite:///5_factbook.db
   sqlite:///factbook.db
Done.


[('Antarctica', 0), ('World', 7256490011)]

In [ ]:
My guess was half-right, half-wrong. The dataset indeed contains an aggregate row for the World. Meanwhile, Antartica, although it is not a country but a continent, is included in the dataset as well. These two should be excluded from the analysis to keep the level of analysis consistent. 

In [69]:
%%sql
SELECT MIN(population) AS min_pop, 
        MAX(population) AS max_pop, 
        MIN(population_growth) AS min_pop_grt,
        MAX(population_growth) AS max_pop_grt
FROM facts
WHERE population != (SELECT MIN(population) FROM facts) AND
population != (SELECT MAX(population) FROM facts)

 * sqlite:///5_factbook.db
   sqlite:///factbook.db
Done.


[(48, 1367485388, 0.0, 4.02)]

Wait, there is a country with a population of 45 people?! This is indeed strange. It's easy to guess which country would have the largest population: China! Let's find out for real.

In [71]:
%%sql
SELECT name, population
FROM facts
WHERE population = (SELECT MIN(population) FROM facts) OR
population = (SELECT MAX(population) FROM facts)

 * sqlite:///5_factbook.db
   sqlite:///factbook.db
Done.


[('Antarctica', 0), ('World', 7256490011)]

In [64]:
%%sql
SELECT name, population, area
FROM facts
WHERE population > (SELECT AVG(population) FROM facts) AND area < (SELECT AVG(area) FROM facts);

 * sqlite:///5_factbook.db
   sqlite:///factbook.db
Done.


[('Bangladesh', 168957745, 148460),
 ('Germany', 80854408, 357022),
 ('Japan', 126919659, 377915),
 ('Philippines', 100998376, 300000),
 ('Thailand', 67976405, 513120),
 ('United Kingdom', 64088222, 243610),
 ('Vietnam', 94348835, 331210)]